# Introduction
<hr style="border:2px solid black"> </hr>

<div class="alert alert-warning">
<font color=black>

**What?** How to Get Reproducible Results with Keras

</font>
</div>

# Import modules
<hr style="border:2px solid black"> </hr>

In [3]:
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error
# Getting rid of the warning messages
import warnings
warnings.filterwarnings("ignore")

# Why do we need to fix the seeder at all?
<hr style="border:2px solid black"> </hr>

<div class="alert alert-info">
<font color=black>

- Neural network algorithms are stochastic hence they make use of randomness.
- Sources of randomness:
    - Randomness in Initialization, such as weights.
    - Randomness in Regularization, such as dropout.
    - Randomness in Layers, such as word embedding.
    - Randomness in Optimization, such as stochastic optimization.
- The consequences is that the same network trained on the same data can produce different results.
- The random initialization allows the network to learn a good approximation for the function being learned. This is done by design.
- **Nevertheless**, there are times when you need the exact same result every time the same network is trained on the same data. Such as for a tutorial, or perhaps operationally.  In this case you have to seed the random number generator so that you can get the same results from the same network on the same data, every time.

</font>
</div>

# Problem demonstration
<hr style="border:2px solid black"> </hr>

<div class="alert alert-info">
<font color=black>

- We'll develop a Multilayer Perceptron model to learn a short sequence of numbers increasing by 0.1 from 0.0 to 0.9. Given 0.0, the model must predict 0.1; given 0.1, the model must output 0.2; and so on.
- We will use a network with 1 input, 10 neurons in the hidden layer, and 1 output. The network will use a mean squared error loss function and will be trained using the efficient ADAM algorithm.
- The network needs about 1,000 epochs to solve this problem effectively, but we will only train it for 100 epochs. This is to ensure we get a model that makes errors when making predictions.
- After the network is trained, we will make predictions on the dataset and print the mean squared error.
- You'll see that at each run the model output different MSE values.

</font>
</div>

In [5]:
# fit MLP to dataset and print error
def fit_model(X, y):
    # design network
    model = Sequential()
    model.add(Dense(10, input_dim=1))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    # fit network
    model.fit(X, y, epochs=100, batch_size=len(X), verbose=0)
    # forecast
    yhat = model.predict(X, verbose=0)
    print("MSE: ", mean_squared_error(y, yhat[:, 0]))


# create sequence
length = 10
sequence = [i/float(length) for i in range(length)]
# create X/y pairs
df = DataFrame(sequence)
df = concat([df.shift(1), df], axis=1)
df.dropna(inplace=True)
# convert to MLP friendly format
values = df.values
X, y = values[:, 0], values[:, 1]
# repeat experiment
repeats = 10
for _ in range(repeats):
    fit_model(X, y)

MSE:  0.0003052151160642255
MSE:  0.046978667876896515
MSE:  0.09629765360491839
MSE:  0.09568989718561771
MSE:  4.03520823553351e-06
MSE:  1.2904058471289534e-05
MSE:  0.0020154389393818506
MSE:  0.07920767230102693
MSE:  0.024187434565275844
MSE:  0.1736525164000876


# Solutions
<hr style="border:2px solid black"> </hr>

## Repeat your experiment

<div class="alert alert-info">
<font color=black>

- The traditional and practical way to address this problem is to run your network many times (**30+**) and use statistics to summarize the performance of your model, and compare your model to other models.
- It is not always possible due to the very long training times of some models.
- **This is the most robust method.**

</font>
</div>

## Seed the Random Number Generator

<div class="alert alert-info">
<font color=black>

- Another solution is to use a fixed seed for the random number generator.
- Keras does get its source of randomness from the NumPy random number generator, so this must be seeded regardless of whether you are using a Theano or TensorFlow backend.
- In addition, TensorFlow has its own random number generator that must also be seeded **immediately after** the NumPy random number generator

</font>
</div>

In [8]:
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [9]:
# fit MLP to dataset and print error
def fit_model(X, y):
    # design network
    model = Sequential()
    model.add(Dense(10, input_dim=1))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    # fit network
    model.fit(X, y, epochs=100, batch_size=len(X), verbose=0)
    # forecast
    yhat = model.predict(X, verbose=0)
    print("MSE: ", mean_squared_error(y, yhat[:, 0]))


# create sequence
length = 10
sequence = [i/float(length) for i in range(length)]
# create X/y pairs
df = DataFrame(sequence)
df = concat([df.shift(1), df], axis=1)
df.dropna(inplace=True)
# convert to MLP friendly format
values = df.values
X, y = values[:, 0], values[:, 1]
# repeat experiment
repeats = 10
for _ in range(repeats):
    fit_model(X, y)

MSE:  0.09197406233894682
MSE:  0.0035952748262702947
MSE:  0.021098260664038686
MSE:  9.942473904910191e-06
MSE:  0.09552536115709402
MSE:  0.20751534552028575
MSE:  0.026981315302370648
MSE:  0.002129335642814133
MSE:  0.08815277144169154
MSE:  0.006517984376780486


# Hold a second! I still get something different!
<hr style="border:2px solid black"> </hr>

<div class="alert alert-info">
<font color=black>

- It could be that:
    - Randomness from a Third-Party Library
    - Randomness from Using the GPU
    - Randomness from a Sophisticated Model

</font>
</div>

# References
<hr style="border:2px solid black"> </hr>

<div class="alert alert-warning">
<font color=black>

- https://machinelearningmastery.com/reproducible-results-neural-networks-keras/

</font>
</div>